In [23]:
from transformers import pipeline
import torch

def test_sentiment(texts, yourmodel):
    """
    Test sentiment analysis model with given texts
    """
    # Create sentiment analyzer pipeline
    analyzer = pipeline(
        "sentiment-analysis",
        model=yourmodel,  # your model from HuggingFace Hub
        tokenizer="bert-base-chinese",
        device=0 if torch.cuda.is_available() else -1
    )

    # Process each text
    for text in texts:
        result = analyzer(text)[0]
        sentiment = "positive" if result['label'] == 'LABEL_1' else "negative"
        print(f"\nText: {text}")
        print(f"Sentiment: {sentiment}")
        print(f"Confidence: {result['score']:.4f}")

# Test with example texts
test_texts = [
    "这家店的菜真香，下次还来！",         # The food is delicious, will come again
    "质量有问题，不推荐购买。",           # Quality issues, not recommended
    "快递很快，包装完整。",               # Fast delivery, good packaging
    "商家态度很不好，生气。",             # Bad merchant attitude, angry
    "非常满意，超出预期。",               # Very satisfied, exceeded expectations
    "难吃到极点，太糟糕了。",             # Extremely bad taste, terrible
    "穿着很舒服，尺码合适。",             # Comfortable to wear, good size
    "卖家服务特别好！",                   # Great service from seller
    "不值这个价钱，后悔买了。",           # Not worth the price, regret buying
    "产品完全是垃圾，气死了。"            # Product is totally garbage, very angry
]

# Run the test
test_sentiment(test_texts, "BarryzZ/sentiment-weibo-100k-fine-tuned-bert-test")

Device set to use cuda:0



Text: 这家店的菜真香，下次还来！
Sentiment: positive
Confidence: 1.0000

Text: 质量有问题，不推荐购买。
Sentiment: positive
Confidence: 1.0000

Text: 快递很快，包装完整。
Sentiment: positive
Confidence: 1.0000

Text: 商家态度很不好，生气。
Sentiment: positive
Confidence: 1.0000

Text: 非常满意，超出预期。
Sentiment: positive
Confidence: 1.0000

Text: 难吃到极点，太糟糕了。
Sentiment: positive
Confidence: 1.0000

Text: 穿着很舒服，尺码合适。
Sentiment: positive
Confidence: 1.0000

Text: 卖家服务特别好！
Sentiment: positive
Confidence: 1.0000

Text: 不值这个价钱，后悔买了。
Sentiment: positive
Confidence: 1.0000

Text: 产品完全是垃圾，气死了。
Sentiment: positive
Confidence: 0.9997


In [1]:
print(1+1)

2


There's clearly an issue with our model's predictions. The model is:
1. Classifying everything as positive (positive sentiment)
2. Doing so with extremely high confidence (nearly 100%)
3. Failing to identify obvious negative sentiments like "难吃到极点" and "产品完全是垃圾"

These issues are likely due to optimization problems rather than data imbalance. Our adjustments focus on:
1. Better monitoring (more frequent evaluation, detailed metrics)
2. Improved efficiency (larger batches, mixed precision)
3. Extended training (more epochs, early stopping)

Let's test the model with these optimized parameters.

In [ ]:
# Initialize model with same configuration
model = BertForSequenceClassification.from_pretrained(
   'bert-base-chinese',
   num_labels=2
)
model = model.to(device)

# Enhanced training arguments
training_args = TrainingArguments(
   output_dir='sentiment-weibo-100k-fine-tuned-bert',
   num_train_epochs=5,                    # Increased from 3 to 5 for better learning
   per_device_train_batch_size=64,        # Doubled for faster training
   per_device_eval_batch_size=128,        # Doubled for faster evaluation
   learning_rate=2e-5,                    # Kept same learning rate
   warmup_ratio=0.1,                      # Added warmup ratio for smoother training
   weight_decay=0.01,                     # For regularization
   logging_dir='./logs',
   logging_steps=100,
   evaluation_strategy="steps",           # Changed to step-based evaluation
   eval_steps=200,                        # More frequent evaluation
   save_strategy="steps",
   save_steps=200,                        # More frequent model saving
   load_best_model_at_end=True,
   metric_for_best_model="f1_avg",        # Using average F1 score to select best model
   push_to_hub=True,
   gradient_accumulation_steps=1,
   fp16=True                              # Added mixed precision training for efficiency
)

# Enhanced metrics computation function
def compute_metrics(pred):
   """
   Compute detailed metrics including class-specific scores
   Returns metrics for both positive and negative classes
   """
   labels = pred.label_ids
   preds = pred.predictions.argmax(-1)

   precision, recall, f1, _ = precision_recall_fscore_support(
       labels,
       preds,
       average=None,
       labels=[0, 1]
   )
   acc = accuracy_score(labels, preds)
   conf_mat = confusion_matrix(labels, preds)

   return {
       'accuracy': acc,
       'f1_neg': f1[0],                   # Added separate F1 scores
       'f1_pos': f1[1],
       'f1_avg': f1.mean(),
       'precision_neg': precision[0],      # Added class-specific precision
       'precision_pos': precision[1],
       'recall_neg': recall[0],           # Added class-specific recall
       'recall_pos': recall[1],
       'confusion_matrix': conf_mat.tolist()  # Added confusion matrix
   }


# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Print dataset statistics before training
print("\nDataset Statistics:")
print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print("\nLabel Distribution:")
print("Training:", pd.Series([d['labels'].item() for d in train_dataset]).value_counts())
print("Testing:", pd.Series([d['labels'].item() for d in test_dataset]).value_counts())

# Start training
trainer.train()

# Evaluate model
eval_results = trainer.evaluate()
print("\nFinal Evaluation Results:")
for metric, value in eval_results.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")


Dataset Statistics:
Training samples: 95990
Test samples: 23998

Label Distribution:
Training: 0    48151
1    47839
Name: count, dtype: int64
Testing: 1    12154
0    11844
Name: count, dtype: int64


Step,Training Loss,Validation Loss,Accuracy,F1 Neg,F1 Pos,F1 Avg,Precision Neg,Precision Pos,Recall Neg,Recall Pos,Confusion Matrix
200,0.132900,0.053297,0.982665,0.982690,0.982641,0.982665,0.968822,0.996952,0.996960,0.968735,"[[11808, 36], [380, 11774]]"
400,0.049400,0.040399,0.983415,0.983476,0.983354,0.983415,0.967489,1.000000,1.000000,0.967254,"[[11844, 0], [398, 11756]]"
600,0.043200,0.050095,0.983915,0.983966,0.983864,0.983915,0.968438,1.000000,1.000000,0.968241,"[[11844, 0], [386, 11768]]"
800,0.039300,0.036415,0.983957,0.984007,0.983907,0.983957,0.968517,1.000000,1.000000,0.968323,"[[11844, 0], [385, 11769]]"
1000,0.040700,0.043751,0.983999,0.984048,0.983949,0.983999,0.968597,1.000000,1.000000,0.968405,"[[11844, 0], [384, 11770]]"
1200,0.040700,0.038093,0.983665,0.983721,0.983609,0.983665,0.967963,1.000000,1.000000,0.967747,"[[11844, 0], [392, 11762]]"
1400,0.038600,0.037408,0.983707,0.983762,0.983652,0.983707,0.968043,1.000000,1.000000,0.967830,"[[11844, 0], [391, 11763]]"
1600,0.042000,0.036397,0.983999,0.984048,0.983949,0.983999,0.968597,1.000000,1.000000,0.968405,"[[11844, 0], [384, 11770]]"


Trainer is attempting to log a value of "[[11808, 36], [380, 11774]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[11844, 0], [398, 11756]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[11844, 0], [386, 11768]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[11844, 0], [385, 11769]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[11844, 0], [384, 11770]]

Trainer is attempting to log a value of "[[11844, 0], [384, 11770]]" of type <class 'list'> for key "eval/confusion_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.



Final Evaluation Results:
eval_loss: 0.0438
eval_accuracy: 0.9840
eval_f1_neg: 0.9840
eval_f1_pos: 0.9839
eval_f1_avg: 0.9840
eval_precision_neg: 0.9686
eval_precision_pos: 1.0000
eval_recall_neg: 1.0000
eval_recall_pos: 0.9684
eval_confusion_matrix: [[11844, 0], [384, 11770]]
eval_runtime: 8.5367
eval_samples_per_second: 2811.1650
eval_steps_per_second: 22.0230
epoch: 1.0667


The model achieved excellent metrics after just one epoch.

In [26]:
# Test with example texts
test_texts = [
    "这家店的菜真香，下次还来！",         # The food is delicious, will come again
    "质量有问题，不推荐。",           # Quality issues, not recommended
    "快递很快，包装完整。",               # Fast delivery, good packaging
    "商家态度不好，生气。",             # Bad merchant attitude, angry
    "非常满意，超出预期。",               # Very satisfied, exceeded expectations
    "难吃到极点，太糟糕了。",             # Extremely bad taste, terrible
    "穿着很舒服，尺码合适。",             # Comfortable to wear, good size
    "卖家服务特别好！",                   # Great service from seller
    "不值这个价钱，后悔买了。",           # Not worth the price, regret buying
    "产品完全是垃圾，气死了。"            # Product is totally garbage, very angry
]

# Run the test
test_sentiment(test_texts, "BarryzZ/sentiment-weibo-100k-fine-tuned-bert")

Device set to use cuda:0



Text: 这家店的菜真香，下次还来！
Sentiment: positive
Confidence: 0.9923

Text: 质量有问题，不推荐。
Sentiment: negative
Confidence: 0.8533

Text: 快递很快，包装完整。
Sentiment: positive
Confidence: 0.9878

Text: 商家态度不好，生气。
Sentiment: negative
Confidence: 0.9732

Text: 非常满意，超出预期。
Sentiment: positive
Confidence: 0.9791

Text: 难吃到极点，太糟糕了。
Sentiment: negative
Confidence: 0.8653

Text: 穿着很舒服，尺码合适。
Sentiment: positive
Confidence: 0.9907

Text: 卖家服务特别好！
Sentiment: positive
Confidence: 0.9922

Text: 不值这个价钱，后悔买了。
Sentiment: negative
Confidence: 0.8147

Text: 产品完全是垃圾，气死了。
Sentiment: negative
Confidence: 0.9863


After parameter optimization, our model shows significant improvements.

 Let's test it with some new scenarios to verify its robustness.

In [27]:
test_texts = [
    # Strong positive / 强烈正面
    "我考上研究生了！",  # I got accepted into graduate school!
    "今天他向我求婚了！",  # He proposed to me today!
    "终于买到梦想的房子",  # Finally bought my dream house
    "中了五百万大奖！",  # Won a 5 million prize!

    # Strong negative / 强烈负面
    "被裁员了，好绝望",  # Got laid off, feeling desperate
    "信任的人背叛我",  # Betrayed by someone I trusted
    "重要的文件全丢了",  # Lost all important documents
    "又被扣工资了，气死",  # Got my salary deducted again, so angry

    # Anger / 愤怒
    "偷我的车，混蛋！",  # Someone stole my car, bastard!
    "骗子公司，我要报警",  # Scam company, I'm calling the police
    "半夜装修，烦死了",  # Renovation at midnight, so annoying
    "商家太坑人了！",  # The merchant is such a ripoff!

    # Pleasant surprise / 惊喜
    "宝宝会走路了！",  # Baby learned to walk!
    "升职加薪啦！",  # Got promoted with a raise!
    "论文发表成功！",  # Paper got published successfully!
    "收到offer了！"  # Received a job offer!
]
# Run the test
test_sentiment(test_texts, "BarryzZ/sentiment-weibo-100k-fine-tuned-bert")

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



Text: 我考上研究生了！
Sentiment: positive
Confidence: 0.8713

Text: 今天他向我求婚了！
Sentiment: positive
Confidence: 0.6087

Text: 终于买到梦想的房子
Sentiment: positive
Confidence: 0.7931

Text: 中了五百万大奖！
Sentiment: positive
Confidence: 0.6070

Text: 被裁员了，好绝望
Sentiment: negative
Confidence: 0.9973

Text: 信任的人背叛我
Sentiment: negative
Confidence: 0.9572

Text: 重要的文件全丢了
Sentiment: negative
Confidence: 0.9941

Text: 又被扣工资了，气死
Sentiment: negative
Confidence: 0.9963

Text: 偷我的车，混蛋！
Sentiment: negative
Confidence: 0.9664

Text: 骗子公司，我要报警
Sentiment: negative
Confidence: 0.9750

Text: 半夜装修，烦死了
Sentiment: negative
Confidence: 0.9906

Text: 商家太坑人了！
Sentiment: negative
Confidence: 0.8367

Text: 宝宝会走路了！
Sentiment: positive
Confidence: 0.9125

Text: 升职加薪啦！
Sentiment: positive
Confidence: 0.9727

Text: 论文发表成功！
Sentiment: positive
Confidence: 0.9998

Text: 收到offer了！
Sentiment: positive
Confidence: 0.7036


The optimized model shows excellent performance in Chinese sentiment analysis. It now correctly identifies both positive and negative sentiments with appropriate confidence levels, while maintaining more moderate confidence for nuanced cases. 